In [1]:
import pandas as pd
from pathlib import Path

# Replace this with your dataset folder seen in Kaggle sidebar
DATASET_FOLDER = "/kaggle/input/exoplanetdataset"  

# Replace with your actual filename
RAW_PATH = Path(DATASET_FOLDER) / "Dataset_40.csv"

RAW_PATH


PosixPath('/kaggle/input/exoplanetdataset/Dataset_40.csv')

In [2]:
import os

print(os.listdir("/kaggle/input/dataset-40-csv"))

# Cell A: load the dataset (exact filename discovered)
import pandas as pd
from pathlib import Path

RAW_PATH = Path("/kaggle/input/dataset-40-csv/Dataset_40.csv.csv")
print("Raw path exists?:", RAW_PATH.exists())
print("Raw path:", RAW_PATH)
# try loading (skip NASA comment lines)
df = pd.read_csv(RAW_PATH, comment='#', low_memory=False)
print("Loaded shape:", df.shape)
df.head(5)


['Dataset_40.csv.csv']
Raw path exists?: True
Raw path: /kaggle/input/dataset-40-csv/Dataset_40.csv.csv
Loaded shape: (39119, 42)


,pl_name,hostname,default_flag,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,soltype,pl_controv_flag,...,ra,decstr,dec,sy_dist,sy_vmag,sy_kmag,sy_gaiamag,rowupdate,pl_pubdate,releasedate
0,11 Com b,11 Com,1,2,1,Radial Velocity,2007,Xinglong Station,Published Confirmed,0,...,185.178779,+17d47m35.71s,17.793252,93.1846,4.72307,2.282,4.44038,2023-09-19,2023-08,2023-09-19
1,11 Com b,11 Com,0,2,1,Radial Velocity,2007,Xinglong Station,Published Confirmed,0,...,185.178779,+17d47m35.71s,17.793252,93.1846,4.72307,2.282,4.44038,2014-05-14,2008-01,2014-05-14
2,11 Com b,11 Com,0,2,1,Radial Velocity,2007,Xinglong Station,Published Confirmed,0,...,185.178779,+17d47m35.71s,17.793252,93.1846,4.72307,2.282,4.44038,2014-07-23,2011-08,2014-07-23
3,11 UMi b,11 UMi,0,1,1,Radial Velocity,2009,Thueringer Landessternwarte Tautenburg,Published Confirmed,0,...,229.274595,+71d49m26.19s,71.823943,125.3210,5.01300,1.939,4.56216,2018-04-25,2009-10,2014-05-14
4,11 UMi b,11 UMi,1,1,1,Radial Velocity,2009,Thueringer Landessternwarte Tautenburg,Published Confirmed,0,...,229.274595,+71d49m26.19s,71.823943,125.3210,5.01300,1.939,4.56216,2018-09-04,2017-03,2018-09-06


In [3]:
print("Rows,Cols:", df.shape)

print("\nFirst 40 column names:")
for i, c in enumerate(list(df.columns)[:40], 1):
    print(f"{i:02d}. {c}")

print("\nData types & non-null counts:")
df.info(verbose=True, show_counts=True)



Rows,Cols: (39119, 42)

First 40 column names:
01. pl_name
02. hostname
03. default_flag
04. sy_snum
05. sy_pnum
06. discoverymethod
07. disc_year
08. disc_facility
09. soltype
10. pl_controv_flag
11. pl_refname
12. pl_orbper
13. pl_orbsmax
14. pl_rade
15. pl_radj
16. pl_bmasse
17. pl_bmassj
18. pl_bmassprov
19. pl_orbeccen
20. pl_insol
21. pl_eqt
22. ttv_flag
23. st_refname
24. st_spectype
25. st_teff
26. st_rad
27. st_mass
28. st_met
29. st_metratio
30. st_logg
31. sy_refname
32. rastr
33. ra
34. decstr
35. dec
36. sy_dist
37. sy_vmag
38. sy_kmag
39. sy_gaiamag
40. rowupdate

Data types & non-null counts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39119 entries, 0 to 39118
Data columns (total 42 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   pl_name          39119 non-null  object 
 1   hostname         39119 non-null  object 
 2   default_flag     39119 non-null  int64  
 3   sy_snum          39119 non-null  int64  


In [4]:
required = {
 'pl_rade':'planet radius (Earth radii)',
 'pl_bmasse':'planet mass (Earth masses)',
 'pl_dens':'planet density',
 'pl_eqt':'equilibrium temperature (K)',
 'pl_orbper':'orbital period (days)',
 'pl_orbsmax':'semi-major axis (AU)',
 'st_spectype':'host star spectral type',
 'st_lum':'star luminosity',
 'st_teff':'star effective temperature (K)',
 'st_metfe':'star metallicity (Fe/H)',
 'sy_dist':'system distance (pc)'
}

print("Required columns present? (True means present)\n")
for k,v in required.items():
    print(f"{k:12} -> {k in df.columns}")

print("\nTop 15 columns by missing count:")
missing = df.isnull().sum().sort_values(ascending=False)
print(missing.head(15))


Required columns present? (True means present)

pl_rade      -> True
pl_bmasse    -> True
pl_dens      -> False
pl_eqt       -> True
pl_orbper    -> True
pl_orbsmax   -> True
st_spectype  -> True
st_lum       -> False
st_teff      -> True
st_metfe     -> False
sy_dist      -> True

Top 15 columns by missing count:
st_spectype     36261
pl_bmassj       32099
pl_bmasse       32098
pl_bmassprov    32098
pl_insol        22010
pl_eqt          21979
pl_orbeccen     20418
pl_orbsmax      17203
st_metratio     14565
st_met          14368
pl_radj         12140
pl_rade         12140
st_logg          8715
st_mass          6056
st_teff          3451
dtype: int64


In [5]:
# MODULE-1 CLEANING PIPELINE

df_clean = df.copy()

# 1. Strip whitespace from column names
df_clean.columns = [c.strip() for c in df_clean.columns]

# 2. Drop fully empty rows
df_clean = df_clean.dropna(how="all")

# 3. Drop exact duplicate rows
df_clean = df_clean.drop_duplicates()

# 4. If multiple parameter sets exist for same planet, keep most recent
if "pl_name" in df_clean.columns and "rowupdate" in df_clean.columns:
    df_clean = df_clean.sort_values("rowupdate").drop_duplicates(subset=["pl_name"], keep="last")

# 5. Create a consistent rename map for key features
rename_map = {
    "pl_rade": "planet_radius_re",
    "pl_bmasse": "planet_mass_me",
    "pl_eqt": "planet_eq_temp_k",
    "pl_orbper": "orbital_period_days",
    "pl_orbsmax": "semi_major_axis_au",
    "st_teff": "star_temp_k",
    "st_met": "star_metallicity",
    "sy_dist": "system_distance_pc"
}

df_clean = df_clean.rename(columns=rename_map)

# 6. Save cleaned dataset (for download)
clean_path = "/kaggle/working/exoplanets_cleaned.csv"
df_clean.to_csv(clean_path, index=False)

print("Cleaned dataset saved:", clean_path)
print("Final cleaned shape:", df_clean.shape)

df_clean.head()


Cleaned dataset saved: /kaggle/working/exoplanets_cleaned.csv
Final cleaned shape: (6052, 42)


,pl_name,hostname,default_flag,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,soltype,pl_controv_flag,...,ra,decstr,dec,system_distance_pc,sy_vmag,sy_kmag,sy_gaiamag,rowupdate,pl_pubdate,releasedate
3122,HD 233604 b,HD 233604,1,1,1,Radial Velocity,2013,McDonald Observatory,Published Confirmed,0,...,137.453505,+53d34m05.53s,53.568202,847.0580,10.29100,7.979,10.10540,2014-05-14,2013-06,2014-05-14
920,GU Psc b,GU Psc,1,1,1,Imaging,2014,Gemini Observatory,Published Confirmed,0,...,18.146485,+17d03m54.01s,17.065003,47.5501,14.14600,9.345,12.96560,2014-05-14,2014-05,2014-05-14
25868,Kepler-38 b,Kepler-38,1,2,1,Transit,2012,Kepler,Published Confirmed,0,...,286.830312,+42d16m44.92s,42.279143,1174.5900,14.01500,12.342,13.90660,2014-05-14,2012-10,2014-05-14
2916,HD 208527 b,HD 208527,1,1,1,Radial Velocity,2012,Bohyunsan Optical Astronomical Observatory,Published Confirmed,0,...,329.099941,+21d14m23.72s,21.239922,312.2020,6.38984,2.224,5.61607,2014-05-14,2013-01,2014-05-14
257,CT Cha b,CT Cha,1,1,1,Imaging,2007,Paranal Observatory,Published Confirmed,0,...,166.037504,-76d27m19.33s,-76.455369,190.7200,12.33500,8.661,11.77330,2014-05-14,2008-11,2014-05-14


In [6]:
data_dict_df = pd.DataFrame({
    "column_name": df_clean.columns,
    "description": [""] * len(df_clean.columns)
})

data_dict_df.to_csv("/kaggle/working/data_dictionary.csv", index=False)

print("Saved data_dictionary.csv")


Saved data_dictionary.csv


In [7]:
missing_after = df_clean.isnull().sum().sort_values(ascending=False)

with open("/kaggle/working/validation_report.txt", "w") as f:
    f.write("Original shape: " + str(df.shape) + "\n")
    f.write("Cleaned shape: " + str(df_clean.shape) + "\n\n")
    f.write("Missing values (top 20):\n")
    f.write(str(missing_after.head(20)))

print("Saved validation_report.txt")


Saved validation_report.txt
